In [1]:
import sympy as sp
import sympy.physics.mechanics as me

sp.init_printing()

In [2]:
# symbols
mb, mm, Ib, Im, l, r, g = sp.symbols('m_b m_m I_b I_m l r g', real=True, positive=True) # Model parameters (stationary)
T1, T2, tau = sp.symbols('T_1 T_2 tau', real=True) # Control inputs
t = sp.symbols('t', real=True) # Time

xb, yb, pb, q = me.dynamicsymbols('x_b y_b p_b q') # Generalized coordinates
xb_dot, yb_dot, pb_dot, q_dot = me.dynamicsymbols('x_b y_b p_b q', 1) # Generalized speeds
xb_ddot, yb_ddot, pb_ddot, q_ddot = me.dynamicsymbols('x_b y_b p_b q', 2) # Generalized accelerations

In [3]:
# Kinetic and potential energy
Kb = 0.5*mb*(xb_dot**2 + yb_dot**2) + 0.5*Ib*(pb_dot**2)
Km = 0.5*mm*(xb_dot**2 + yb_dot**2) + 0.5*mm*(0.5*l*(q_dot+pb_dot))**2 + 0.5*Im*(q_dot+pb_dot)**2
T = Kb + Km
V = mb*g*yb + mm*g*(yb - 0.5*l*sp.cos(q))
# Euler-Lagrange equations
L = T - V
eqn_lhs = sp.Matrix([sp.diff(sp.diff(L, xb_dot), 't')-sp.diff(L, xb),
                sp.diff(sp.diff(L, yb_dot), 't')-sp.diff(L, yb),
                sp.diff(sp.diff(L, pb_dot), 't')-sp.diff(L, pb),
                sp.diff(sp.diff(L, q_dot), 't')-sp.diff(L, q)])
eqn_lhs = sp.expand(eqn_lhs)
display(eqn_lhs)

F = sp.Matrix([sp.sin(pb)*(T1+T2), sp.cos(pb)*(T1+T2), -r*T1+r*T2, tau])



⎡                                           2                    2             ↪
⎢                                          d                    d              ↪
⎢                                  1.0⋅m_b⋅───(x_b(t)) + 1.0⋅mₘ⋅───(x_b(t))    ↪
⎢                                            2                    2            ↪
⎢                                          dt                   dt             ↪
⎢                                                                              ↪
⎢                                                  2                    2      ↪
⎢                                                 d                    d       ↪
⎢                          g⋅m_b + g⋅mₘ + 1.0⋅m_b⋅───(y_b(t)) + 1.0⋅mₘ⋅───(y_b ↪
⎢                                                   2                    2     ↪
⎢                                                 dt                   dt      ↪
⎢                                                                              ↪
⎢         2                 

In [4]:
# Generalized coordinates and accelerations for M, C, G isolation
xis = [xb, yb, pb, q]
xi_dots = [xb_dot, yb_dot, pb_dot, q_dot]
xi_ddots = [xb_ddot, yb_ddot, pb_ddot, q_ddot]

# Isolate M, C, G from the equations
M = sp.zeros(len(xis), len(xis))  # Mass matrix
C = sp.zeros(len(xis), len(xis))  # Coriolis matrix
G = sp.zeros(len(xis), 1)  # Gravity vector

# Iterate over the equations to isolate M, C, G based on their linear dependence
for i, eq in enumerate(eqn_lhs):
    # Extract M
    for j, xi_ddot in enumerate(xi_ddots):
        M[i, j] = sp.simplify(eq.coeff(xi_ddot))
    
    # Subtract the mass-related terms from the equation to find the rest
    eq_residual = eq - sum(M[i, j] * xi_ddots[j] for j in range(len(xi_ddots)))
    
    # Extract C
    for j, xi_dot in enumerate(xi_dots):
        C[i, j] = sp.simplify(eq_residual.coeff(xi_dot))
    
    # Subtract the Coriolis-related terms to find the gravity vector
    eq_residual -= sum(C[i, j] * xi_dots[j] for j in range(len(xi_dots)))
    
    # Extract G
    G[i, 0] = sp.simplify(eq_residual)

# Display results
print("Mass Matrix (M):")
display(M)

print("\nCoriolis Matrix (C):")
display(C)

print("\nGravity Vector (G):")
display(G)

print("\nControl Force Vector (F):")
display(F)

print("\nGeneralized accelerations (xi_ddots):")
EOM = M.inv()*(F - C*sp.Matrix(xi_dots) - G)
display(EOM)

Mass Matrix (M):


⎡1.0⋅m_b + 1.0⋅mₘ         0                        0                         0 ↪
⎢                                                                              ↪
⎢       0          1.0⋅m_b + 1.0⋅mₘ                0                         0 ↪
⎢                                                                              ↪
⎢                                                             2                ↪
⎢       0                 0          1.0⋅I_b + 1.0⋅Iₘ + 0.25⋅l ⋅mₘ  1.0⋅Iₘ + 0 ↪
⎢                                                                              ↪
⎢                                                        2                     ↪
⎣       0                 0               1.0⋅Iₘ + 0.25⋅l ⋅mₘ       1.0⋅Iₘ + 0 ↪

↪          ⎤
↪          ⎥
↪          ⎥
↪          ⎥
↪      2   ⎥
↪ .25⋅l ⋅mₘ⎥
↪          ⎥
↪      2   ⎥
↪ .25⋅l ⋅mₘ⎦


Coriolis Matrix (C):


⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦


Gravity Vector (G):


⎡         0          ⎤
⎢                    ⎥
⎢    g⋅(m_b + mₘ)    ⎥
⎢                    ⎥
⎢         0          ⎥
⎢                    ⎥
⎣0.5⋅g⋅l⋅mₘ⋅sin(q(t))⎦


Control Force Vector (F):


⎡(T₁ + T₂)⋅sin(p_b(t))⎤
⎢                     ⎥
⎢(T₁ + T₂)⋅cos(p_b(t))⎥
⎢                     ⎥
⎢    -T₁⋅r + T₂⋅r     ⎥
⎢                     ⎥
⎣          τ          ⎦


Generalized accelerations (xi_ddots):


⎡                           0.25⋅(T₁ + T₂)⋅sin(p_b(t))                         ↪
⎢                           ──────────────────────────                         ↪
⎢                               0.25⋅m_b + 0.25⋅mₘ                             ↪
⎢                                                                              ↪
⎢                  0.25⋅(-g⋅(m_b + mₘ) + (T₁ + T₂)⋅cos(p_b(t)))                ↪
⎢                  ────────────────────────────────────────────                ↪
⎢                               0.25⋅m_b + 0.25⋅mₘ                             ↪
⎢                                                                              ↪
⎢              1.0⋅(-T₁⋅r + T₂⋅r)   1.0⋅(-0.5⋅g⋅l⋅mₘ⋅sin(q(t)) + τ)            ↪
⎢              ────────────────── - ───────────────────────────────            ↪
⎢                     I_b                         I_b                          ↪
⎢                                                                              ↪
⎢                           

In [5]:
# Turn into a function of the form x_dot = f(x, u)
x = sp.Matrix([xb, yb, pb, q, xb_dot, yb_dot, pb_dot, q_dot])
u = sp.Matrix([T1, T2, tau])
p = sp.Matrix([mb, mm, Ib, Im, l, r, g])
f = sp.Matrix([xb_dot, yb_dot, pb_dot, q_dot, EOM])
# Substitute all the time-dependent variables with symbols
f = f.subs({xb: sp.symbols('x_b', real=True, positive=True),
            yb: sp.symbols('y_b', real=True, positive=True),
            pb: sp.symbols('p_b', real=True, positive=True),
            q: sp.symbols('q', real=True, positive=True),
            xb_dot: sp.symbols('xb_dot', real=True, positive=True),
            yb_dot: sp.symbols('yb_dot', real=True, positive=True),
            pb_dot: sp.symbols('pb_dot', real=True, positive=True),
            q_dot: sp.symbols('q_dot', real=True, positive=True)})
display(f)

# Save the model to disk
model_name = '2d_simple_aerial_manipulator'
import pickle
with open(f'{model_name}.pkl', 'wb') as file:
    pickle.dump(f, file)


⎡                                   xb_dot                                    ⎤
⎢                                                                             ⎥
⎢                                   yb_dot                                    ⎥
⎢                                                                             ⎥
⎢                                   pb_dot                                    ⎥
⎢                                                                             ⎥
⎢                                    q_dot                                    ⎥
⎢                                                                             ⎥
⎢                           0.25⋅(T₁ + T₂)⋅sin(p_b)                           ⎥
⎢                           ───────────────────────                           ⎥
⎢                             0.25⋅m_b + 0.25⋅mₘ                              ⎥
⎢                                                                             ⎥
⎢                  0.25⋅(-g⋅(m_b + mₘ) +